In [5]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from math import factorial
from scipy import stats as st

In [6]:
df = pd.read_csv('games.csv') 